# Pattern Recognition - Apple Stock 2023-2025

Testing pattern recognition indicators: Pivots, Hammer, RSI Divergence.

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
from pathlib import Path
sys.path.insert(0, str(Path('../..').resolve()))

from indicators.pattern import find_pivots, calculate_hammer, detect_rsi_divergence
from indicators.momentum import calculate_rsi

In [ ]:
# Fetch Apple data
df = yf.download('AAPL', start='2023-01-01', end='2025-10-01', auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

## 1. Pivot Points

In [ ]:
pivot_high, pivot_low = find_pivots(df, lb=8, rb=13, use_close=True)
df['PivotHigh'] = pivot_high
df['PivotLow'] = pivot_low

fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1)))

# Pivot highs
pivot_highs = df[df['PivotHigh'] == 1]
fig.add_trace(go.Scatter(x=pivot_highs['date'], y=pivot_highs['close'],
                        mode='markers', name='Pivot High',
                        marker=dict(color='red', size=10, symbol='triangle-down')))

# Pivot lows
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='green', size=10, symbol='triangle-up')))

fig.update_layout(height=600, title_text="Pivot Points Detection (lb=8, rb=13)")
fig.show()

print(f"Pivot Highs found: {(df['PivotHigh'] == 1).sum()}")
print(f"Pivot Lows found: {(df['PivotLow'] == 1).sum()}")

## 2. Hammer Candlestick Pattern

In [ ]:
df['hammer'] = calculate_hammer(df)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price with Hammer Patterns', 'Hammer Signal'))

# Price
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black')), row=1, col=1)

# Mark hammer patterns on price
hammers = df[df['hammer'] == 100]
fig.add_trace(go.Scatter(x=hammers['date'], y=hammers['close'],
                        mode='markers', name='Hammer',
                        marker=dict(color='orange', size=10, symbol='star')), row=1, col=1)

# Hammer signal
fig.add_trace(go.Scatter(x=df['date'], y=df['hammer'], name='Hammer',
                        fill='tozeroy', line=dict(color='orange')), row=2, col=1)

fig.update_layout(height=600, title_text="Hammer Candlestick Pattern (Bullish Reversal)")
fig.show()

print(f"Hammer patterns found: {(df['hammer'] == 100).sum()}")

## 3. RSI Divergence at Pivot Lows

In [ ]:
# Calculate RSI first
df['RSI'] = calculate_rsi(df, period=14)

# Detect divergence
df = detect_rsi_divergence(df)

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=('Price with Divergence', 'RSI', 'Divergence Signal'))

# Price with pivot lows
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black')), row=1, col=1)
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='blue', size=8)), row=1, col=1)

# Mark divergence points
divergences = df[df['RSI_Divergence'] == 1]
fig.add_trace(go.Scatter(x=divergences['date'], y=divergences['close'],
                        mode='markers', name='Bullish Divergence',
                        marker=dict(color='green', size=12, symbol='star')), row=1, col=1)

# RSI
fig.add_trace(go.Scatter(x=df['date'], y=df['RSI'], name='RSI',
                        line=dict(color='blue')), row=2, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="red", row=2, col=1)

# Divergence signal
fig.add_trace(go.Scatter(x=df['date'], y=df['RSI_Divergence'], name='Divergence',
                        fill='tozeroy', line=dict(color='green')), row=3, col=1)

fig.update_layout(height=800, title_text="RSI Bullish Divergence Detection")
fig.show()

print(f"Bullish divergences found: {(df['RSI_Divergence'] == 1).sum()}")
print("Bullish divergence = Lower price but Higher RSI at pivot lows")

## Combined View - All Pattern Indicators

In [ ]:
fig = go.Figure()

# Price line
fig.add_trace(go.Scatter(x=df['date'], y=df['close'], name='Close',
                        line=dict(color='black', width=1.5)))

# Pivot highs (red triangles)
pivot_highs = df[df['PivotHigh'] == 1]
fig.add_trace(go.Scatter(x=pivot_highs['date'], y=pivot_highs['close'],
                        mode='markers', name='Pivot High',
                        marker=dict(color='red', size=10, symbol='triangle-down')))

# Pivot lows (green triangles)
pivot_lows = df[df['PivotLow'] == 1]
fig.add_trace(go.Scatter(x=pivot_lows['date'], y=pivot_lows['close'],
                        mode='markers', name='Pivot Low',
                        marker=dict(color='green', size=10, symbol='triangle-up')))

# Hammer patterns (orange stars)
hammers = df[df['hammer'] == 100]
fig.add_trace(go.Scatter(x=hammers['date'], y=hammers['close'],
                        mode='markers', name='Hammer',
                        marker=dict(color='orange', size=12, symbol='star')))

# RSI divergences (large green circles)
divergences = df[df['RSI_Divergence'] == 1]
fig.add_trace(go.Scatter(x=divergences['date'], y=divergences['close'],
                        mode='markers', name='RSI Divergence',
                        marker=dict(color='lime', size=15, symbol='circle',
                                  line=dict(color='darkgreen', width=2))))

fig.update_layout(height=700, title_text="Pattern Recognition - Complete Overview",
                 xaxis_title="Date", yaxis_title="Price")
fig.show()

print("\n=== Pattern Summary ===")
print(f"Pivot Highs: {(df['PivotHigh'] == 1).sum()}")
print(f"Pivot Lows: {(df['PivotLow'] == 1).sum()}")
print(f"Hammer Patterns: {(df['hammer'] == 100).sum()}")
print(f"RSI Bullish Divergences: {(df['RSI_Divergence'] == 1).sum()}")
print("\nPivots identify local extremes for ML labeling.")
print("Hammers signal potential reversals.")
print("Divergences show momentum/price disagreement.")